In [1]:
import json
import os
train_data_path = 'data/train/2025-04-11'
train_file_paths = list()
skip_files = ['stopwords']

for root, dirs, files in os.walk(train_data_path):
    for file in files:
        if file not in skip_files:
            train_file_paths.append(f'{train_data_path}/{file}')

print(train_file_paths)

[]


In [2]:
from src.util.read_file import read_file

encoding = 'utf-8'
train_novels = list[tuple]()

for file_path in train_file_paths:
    try:
        train_novels.append((file_path, read_file(file_path, encoding)))
    except Exception:
        print('Decode Error ', file_path)

with open('data/stopwords', encoding='utf-8') as _f:
    stopwords = _f.readlines()
if len(train_novels) > 0:
    print(str(train_novels[0])[:512], '...')
print(stopwords[:128])

['!\n', '"\n', '#\n', '$\n', '%\n', '&\n', "'\n", '(\n', ')\n', '*\n', '+\n', ',\n', '-\n', '--\n', '.\n', '..\n', '...\n', '......\n', '...................\n', './\n', '.一\n', '.数\n', '.日\n', '/\n', '//\n', '0\n', '1\n', '2\n', '3\n', '4\n', '5\n', '6\n', '7\n', '8\n', '9\n', ':\n', '://\n', '::\n', ';\n', '<\n', '=\n', '>\n', '>>\n', '?\n', '@\n', 'A\n', 'Lex\n', '[\n', '\\\n', ']\n', '^\n', '_\n', '`\n', 'exp\n', 'sub\n', 'sup\n', '|\n', '}\n', '~\n', '~~~~\n', '·\n', '×\n', '×××\n', 'Δ\n', 'Ψ\n', 'γ\n', 'μ\n', 'φ\n', 'φ．\n', 'В\n', '—\n', '——\n', '———\n', '‘\n', '’\n', '’‘\n', '“\n', '”\n', '”，\n', '…\n', '……\n', '…………………………………………………③\n', '′∈\n', '′｜\n', '℃\n', 'Ⅲ\n', '↑\n', '→\n', '∈［\n', '∪φ∈\n', '≈\n', '①\n', '②\n', '②ｃ\n', '③\n', '③］\n', '④\n', '⑤\n', '⑥\n', '⑦\n', '⑧\n', '⑨\n', '⑩\n', '──\n', '■\n', '▲\n', '\u3000\n', '、\n', '。\n', '〈\n', '〉\n', '《\n', '》\n', '》），\n', '」\n', '『\n', '』\n', '【\n', '】\n', '〔\n', '〕\n', '〕〔\n', '㈧\n', '一\n', '一.\n', '一一\n', '一下\n', '一个\n']


In [3]:
from src.util.preprocess import split_words

with open('data/seg.json', 'r', encoding='utf-8') as _f:
    train_seg_lists = json.load(_f)
    
if len(train_seg_lists) != len(train_novels):
    for novel in train_novels[len(train_seg_lists):]:
        train_seg_lists.append((novel[0], split_words(novel[1], stopwords)))
        with open('data/seg.json', mode='w', encoding='utf-8') as _f:
            json.dump(train_seg_lists, _f, ensure_ascii=False)
    
print(train_seg_lists[0])

['data/train/2025-04-11/1-相亲闪婚后，老公竟是千亿财阀.txt', ['闪婚', '摆架子', '书名', '相亲', '婚后', '老公', '竟是', '财阀', '作者', '本章', '字数', '更新', '时间', '今天', '晚上', '外面', '吃个', '回来', '舅舅', '舅妈', '发疯', '电话', '那头', '舅舅', '的话', '说完', '紧跟着', '传来', '舅妈', '尖锐', '声音', '发疯', '贱种', '侄女', '今年', '读书', '嫁人', '我家', '发疯', '现在', '公司', '遇到', '这么', '危机', '不管', '必须', '介绍', '准备', '开口', '电话', '匆忙', '挂断', '无声', '地叹', '口气', '看着', '自己', '刚发', '奖学金', '一份', '没动', '舅舅', '过去', '这样', '情况', '基本上', '每个', '星期', '出现', '自从', '妈妈', '去世', '舅舅', '回家', '因为', '舅妈', '喜欢', '只能', '这个', '小心翼翼', '生活', '直到', '考上', '文物', '修复', '研究生', '前些年', '舅舅', '学费', '奖学金', '干净', '想着', '这个', '无论如何', '搬走', '苦于', '舅舅', '一直', '同意', '舅妈', '因此', '一直', '吵架', '唯一', '想到', '能够', '正大光明', '搬走', '舅舅', '不会', '担心', '借口', '就是', '结婚', '现在', '正在', '结婚', '路上', '看着', '窗外', '逐渐', '停缓', '树影', '出现', '眼前', '民政局', '三个', '大字', '深吸', '一口气', '下车', '这里', '苍老', '中气', '十足', '声音', '响起', '一转眼', '看到', '穿着', '简单', '眼眸', '说来', '相遇', '现在', '就读', '清美', '大学', '作为', '特邀', '高龄', '教授', '演讲', '因为', '朴素', '险些', 

In [7]:
train_documents = [x[1] for x in train_seg_lists]

In [6]:
from gensim import corpora

dictionary = corpora.Dictionary(train_documents)
corpus = [dictionary.doc2bow(x) for x in train_documents]

print(corpus[:16])

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

num_topics = 10

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=64,
    random_state=32
)


cm = CoherenceModel(model=lda_model, texts=train_documents, coherence='u_mass')
coherence = cm.get_coherence()
print(f"Coherence={coherence}")

In [ ]:
pyLDAvis.display(gensimvis.prepare(lda_model, corpus, dictionary))

In [ ]:
# save model
lda_model.save('model/lda/lda_model.pkl')